# BUILDING DATAFRAME

# 1.0 Import the required libraries first

In [1]:
!pip install geopy
!pip install beautifulsoup4
!pip install html5lib
!pip install lxml

In [2]:
import pandas as pd
import numpy as np
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print('Libraries imported.')

Libraries imported.


In [3]:
from bs4 import BeautifulSoup

# 2.0 Extract the table from the page using Pandas

In [4]:
from pandas.io.html import read_html
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikitables = read_html(page, attrs={"class":"sortable"})
print("Extracted {num} wikitables".format(num=len(wikitables)))

Extracted 1 wikitables


In [5]:
table = wikitables[0]

In [7]:
table.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# 3.0 DATA PRE-PROCESSING

In [8]:
Table_data = table.rename(columns = {'Postcode':'Postal Code', 'Neighbourhood':'Neighborhood'})

In [10]:
Table_data.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## 3.1 REMOVING BOROUGHS CONTAINING "NOT ASSIGNED"

In [11]:
Table_data = Table_data[Table_data.Borough != 'Not assigned']

In [12]:
Table_data.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Not assigned
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


## 3.2 Pre-processing cells that has a borough but 'Not assigned' neighborhood, then the neighborhood will be the same as the borough.

In [13]:
Table_data['Neighborhood'] = Table_data['Neighborhood'].replace('Not assigned', Table_data['Borough'])

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
Table_data.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


## 3.3 COMBINING NEIGHBORHOODS WITH SAME POSTAL CODES

In [16]:
Table_data_1 = Table_data.groupby(('Postal Code','Borough'))['Neighborhood'].apply(lambda x: ', '.join(x.astype(str))).reset_index()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.


In [17]:
Table_data_1.head(10)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


# 4.0 SHAPE OF TABLE

In [18]:
Table_data_1.shape

(103, 3)

# SECOND PART OF PROJECT

## Download Geopsatial data

In [19]:
!wget -q -O 'Geospatial_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [21]:
t=pd.read_csv('Geospatial_data.csv')

In [23]:
t.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge Dataframe with Geospatial data

In [24]:
New_Table = pd.merge(Table_data_1, t, on = 'Postal Code', how = 'inner')

In [25]:
New_Table.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# The neighborhoods in Toronto

In [26]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude1 = location.latitude
longitude1 = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude1, longitude1))

The geograpical coordinate of Downtown Toronto are 43.653963, -79.387207.


In [27]:
map_Toronto = folium.Map(location=[43.653963, -79.387207], zoom_start=10)

# add markers to map
for lat, lng, Borough, Neighborhood in zip(New_Table['Latitude'], New_Table['Longitude'], New_Table['Borough'], New_Table['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

## Exploring Boroughs containing Downtown Toronto close to the Billy Bishop Toronto City Airport

In [28]:
Borough_toronto_data = New_Table[New_Table['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Borough_toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [29]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [30]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[43.6563221, -79.3809161], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Borough_toronto_data['Latitude'], Borough_toronto_data['Longitude'], Borough_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto

## Exploring Boroughs containing North York close to the Downsview Airport

In [31]:
address = 'North York, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude_ = location.latitude
longitude_ = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude_, longitude_))

The geograpical coordinate of Downtown Toronto are 43.7543263, -79.44911696639593.


In [32]:
Borough_northyork_data = New_Table[New_Table['Borough'] == 'North York'].reset_index(drop=True)
Borough_northyork_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493


In [33]:
# create map of Downtown Toronto using latitude and longitude values
map_NorthYork = folium.Map(location=[43.7543263, -79.44911696639593], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Borough_northyork_data['Latitude'], Borough_northyork_data['Longitude'], Borough_northyork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NorthYork)  
    
map_NorthYork

# Define Foursquare Credentials and Version

In [34]:
CLIENT_ID = 'KLL5RCHVNHSCMQTVQYZOL3KC4ZRBPJJQDMPHIPPGGK5WXOIH'
CLIENT_SECRET = '1AKYF0PM3L2KPSVQGKBXTSACXN4PLWSVUZJALCCX2LGHNSYR'
VERSION = '20180605'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KLL5RCHVNHSCMQTVQYZOL3KC4ZRBPJJQDMPHIPPGGK5WXOIH
CLIENT_SECRET:1AKYF0PM3L2KPSVQGKBXTSACXN4PLWSVUZJALCCX2LGHNSYR


# Choosing a venue from Downtown Neighborhood

In [ ]:
Borough_toronto_data.loc[5, 'Neighborhood']

In [35]:
neighborhood_latitude = Borough_toronto_data.loc[5, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Borough_toronto_data.loc[5, 'Longitude'] # neighborhood longitude value

neighborhood_name = Borough_toronto_data.loc[5, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of St. James Town are 43.6514939, -79.3754179.


## Now, let's get the top 100 venues that are in Rosedale within a radius of 500 meters.

In [37]:
LIMIT = 100
radius = 500 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=KLL5RCHVNHSCMQTVQYZOL3KC4ZRBPJJQDMPHIPPGGK5WXOIH&client_secret=1AKYF0PM3L2KPSVQGKBXTSACXN4PLWSVUZJALCCX2LGHNSYR&v=20180605&ll=43.6514939,-79.3754179&radius=500&limit=100'

In [38]:
results = requests.get(url).json()

In [39]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [40]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Gyu-Kaku Japanese BBQ,Japanese Restaurant,43.651422,-79.375047
1,GoodLife Fitness Toronto 137 Yonge Street,Gym,43.651242,-79.378068
2,Terroni,Italian Restaurant,43.650927,-79.375602
3,GEORGE Restaurant,Restaurant,43.653346,-79.374445
4,Fahrenheit Coffee,Coffee Shop,43.652384,-79.372719


In [41]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## 2. Explore Neighborhoods in DOWNTOWN TORONTO

In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# LIST OF VENUES IN DOWNTOWN TORONTO

In [43]:
Borough_toronto_data_venues = getNearbyVenues(names=Borough_toronto_data['Neighborhood'],
                                   latitudes=Borough_toronto_data['Latitude'],
                                   longitudes=Borough_toronto_data['Longitude']
                                  )

Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Queen's Park


In [44]:
print(Borough_toronto_data_venues.shape)
Borough_toronto_data_venues.head()

(1312, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"Cabbagetown, St. James Town",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


In [45]:
Borough_toronto_data_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",18,18,18,18,18,18
"Cabbagetown, St. James Town",47,47,47,47,47,47
Central Bay Street,82,82,82,82,82,82
"Chinatown, Grange Park, Kensington Market",87,87,87,87,87,87
Christie,17,17,17,17,17,17
Church and Wellesley,86,86,86,86,86,86
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


In [46]:
print('There are {} uniques categories.'.format(len(Borough_toronto_data_venues['Venue Category'].unique())))

There are 206 uniques categories.


## 3. Analyze Each Neighborhood

In [47]:
# one hot encoding
Downtown_onehot = pd.get_dummies(Borough_toronto_data_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Downtown_onehot['Neighborhood'] = Borough_toronto_data_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Downtown_onehot.columns[-1]] + list(Downtown_onehot.columns[:-1])
Downtown_onehot = Downtown_onehot[fixed_columns]

Downtown_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
Downtown_onehot.shape

(1312, 206)

## Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [49]:
Downtown_grouped = Downtown_onehot.groupby('Neighborhood').mean().reset_index()
Downtown_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,...,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.01
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.00
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,Central Bay Street,0.012195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012195,...,0.000000,0.00,0.00,0.00,0.012195,0.000000,0.000000,0.012195,0.000000,0.00
5,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.045977,0.000000,0.057471,0.011494,0.000000,0.00
6,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,Church and Wellesley,0.011628,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,...,0.011628,0.00,0.00,0.00,0.000000,0.000000,0.011628,0.000000,0.011628,0.00
8,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,...,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.00
9,"Design Exchange, Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,...,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.00


## Let's print each neighborhood along with the top 5 most common venues

In [50]:
num_top_venues = 5

for hood in Downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Downtown_grouped[Downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
            venue  freq
0     Coffee Shop  0.07
1            Café  0.04
2      Steakhouse  0.04
3             Bar  0.04
4  Cosmetics Shop  0.03


----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2      Beer Bar  0.04
3          Café  0.04
4        Bakery  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.17
1    Airport Lounge  0.11
2  Airport Terminal  0.11
3             Plane  0.06
4   Harbor / Marina  0.06


----Cabbagetown, St. James Town----
                venue  freq
0         Coffee Shop  0.09
1          Restaurant  0.06
2                Café  0.06
3  Italian Restaurant  0.04
4              Bakery  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1  Italian Restaurant  0.05
2        Burger Joint  0.04
3      Ice Cream Shop  0.04
4           Juice Bar

In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [52]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Downtown_grouped['Neighborhood']

for ind in np.arange(Downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Restaurant,Hotel,Asian Restaurant,Cosmetics Shop,Thai Restaurant,Burger Joint
1,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Cheese Shop,Seafood Restaurant,Farmers Market,Bakery,Steakhouse,Café,Greek Restaurant
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Plane,Harbor / Marina,Sculpture Garden,Boat or Ferry,Rental Car Location,Bar,Coffee Shop
3,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Café,Pizza Place,Italian Restaurant,Chinese Restaurant,Bakery,Park,Pub,Butcher
4,Central Bay Street,Coffee Shop,Italian Restaurant,Ice Cream Shop,Café,Juice Bar,Japanese Restaurant,Burger Joint,Sandwich Place,Bubble Tea Shop,Bar


## 4. Cluster Neighborhoods

In [53]:
# set number of clusters
kclusters = 5

Downtown_grouped_clustering = Downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 0, 4, 0, 0, 0], dtype=int32)

In [54]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Downtown_merged = Borough_toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Downtown_merged = Downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Downtown_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,0,Coffee Shop,Restaurant,Café,Pizza Place,Italian Restaurant,Chinese Restaurant,Bakery,Park,Pub,Butcher
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Gym,Pub,Men's Store,Mediterranean Restaurant,Hotel
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,3,Coffee Shop,Bakery,Park,Pub,Mexican Restaurant,Café,Restaurant,Breakfast Spot,Event Space,Spa
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Italian Restaurant,Restaurant,Lingerie Store,Ramen Restaurant


In [55]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Downtown_merged['Latitude'], Downtown_merged['Longitude'], Downtown_merged['Neighborhood'], Downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

Now, examining each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.

#### Cluster 1

In [56]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 0, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Pizza Place,Italian Restaurant,Chinese Restaurant,Bakery,Park,Pub,Butcher
2,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Gym,Pub,Men's Store,Mediterranean Restaurant,Hotel
4,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Italian Restaurant,Restaurant,Lingerie Store,Ramen Restaurant
5,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Cocktail Bar,Hotel,Bakery,Beer Bar,Breakfast Spot,Italian Restaurant,Cosmetics Shop
6,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Beer Bar,Cheese Shop,Seafood Restaurant,Farmers Market,Bakery,Steakhouse,Café,Greek Restaurant
7,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Ice Cream Shop,Café,Juice Bar,Japanese Restaurant,Burger Joint,Sandwich Place,Bubble Tea Shop,Bar
8,Downtown Toronto,0,Coffee Shop,Café,Bar,Steakhouse,Restaurant,Hotel,Asian Restaurant,Cosmetics Shop,Thai Restaurant,Burger Joint
9,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Brewery,Scenic Lookout,Restaurant,Italian Restaurant,Fried Chicken Joint,History Museum
10,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Bar,Gastropub,Seafood Restaurant,Steakhouse,Italian Restaurant,Salad Place
11,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Gym,Seafood Restaurant,Japanese Restaurant,Steakhouse,Deli / Bodega,Italian Restaurant


#### Cluster 2

In [57]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 1, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Park,Playground,Trail,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner


#### Cluster 3

In [58]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 2, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,2,Airport Service,Airport Lounge,Airport Terminal,Plane,Harbor / Marina,Sculpture Garden,Boat or Ferry,Rental Car Location,Bar,Coffee Shop


#### Cluster 4

In [59]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 3, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Toronto,3,Coffee Shop,Bakery,Park,Pub,Mexican Restaurant,Café,Restaurant,Breakfast Spot,Event Space,Spa
18,Downtown Toronto,3,Coffee Shop,Gym,Park,Diner,Portuguese Restaurant,Nightclub,Mexican Restaurant,Juice Bar,Italian Restaurant,Hobby Shop


#### Cluster 5

In [60]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 4, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,4,Grocery Store,Café,Park,Coffee Shop,Candy Store,Diner,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports


# Choosing a venue from Northyork Neighborhood

In [61]:
Borough_northyork_data.loc[5, 'Neighborhood']

'Willowdale South'

In [62]:
neighborhood_latitude1 = Borough_northyork_data.loc[5, 'Latitude'] # neighborhood latitude value
neighborhood_longitude1 = Borough_northyork_data.loc[5, 'Longitude'] # neighborhood longitude value

neighborhood_name1 = Borough_northyork_data.loc[5, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name1, 
                                                               neighborhood_latitude1, 
                                                               neighborhood_longitude1))

Latitude and longitude values of Willowdale South are 43.7701199, -79.40849279999999.


## Now, let's get the top 100 venues that are in Willowdale South within a radius of 500 meters.

In [63]:
LIMIT = 100
radius = 500 
url1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude1, 
    neighborhood_longitude1, 
    radius, 
    LIMIT)
url1 

'https://api.foursquare.com/v2/venues/explore?&client_id=KLL5RCHVNHSCMQTVQYZOL3KC4ZRBPJJQDMPHIPPGGK5WXOIH&client_secret=1AKYF0PM3L2KPSVQGKBXTSACXN4PLWSVUZJALCCX2LGHNSYR&v=20180605&ll=43.7701199,-79.40849279999999&radius=500&limit=100'

In [64]:
results1 = requests.get(url1).json()

In [65]:
venues1 = results1['response']['groups'][0]['items']
    
nearby_venues1 = json_normalize(venues1) # flatten JSON

# filter columns
filtered_columns1 = ['venue.name1', 'venue.categories1', 'venue.location.lat1', 'venue.location.lng1']
nearby_venues1 =nearby_venues1.loc[:, filtered_columns]

# filter the category for each row
nearby_venues1['venue.categories'] = nearby_venues1.apply(get_category_type, axis=1)

# clean columns
nearby_venues1.columns = [col.split(".")[-1] for col in nearby_venues1.columns]

nearby_venues1.head()

,name,categories,lat,lng
0,Loblaws,Grocery Store,43.768722,-79.412101
1,Konjiki Ramen,Ramen Restaurant,43.766998,-79.412222
2,The Keg,Steakhouse,43.766579,-79.412131
3,Cineplex Cinemas Empress Walk,Movie Theater,43.768625,-79.412613
4,Starbucks,Coffee Shop,43.768353,-79.413046


In [66]:
print('{} venues were returned by Foursquare.'.format(nearby_venues1.shape[0]))

35 venues were returned by Foursquare.


## 2. Explore Neighborhoods in NORTH YORK

In [67]:
def getNearbyVenues1(names, latitudes, longitudes, radius=500):
    
    venues_list1=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results1 = requests.get(url1).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list1.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results1])

    nearby_venues1 = pd.DataFrame([item for venue_list1 in venues_list1 for item in venue_list1])
    nearby_venues1.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues1)

# LIST OF VENUES IN NORTH YORK

In [68]:
Borough_northyork_data_venues = getNearbyVenues1(names=Borough_northyork_data['Neighborhood'],
                                   latitudes=Borough_northyork_data['Latitude'],
                                   longitudes=Borough_northyork_data['Longitude']
                                  )

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Heights, Lawrence Manor
Glencairn
Downsview, North Park, Upwood Park
Humber Summit
Emery, Humberlea


In [69]:
print(Borough_northyork_data_venues.shape)
Borough_northyork_data_venues.head()

(840, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Loblaws,43.768722,-79.412101,Grocery Store
1,Hillcrest Village,43.803762,-79.363452,Konjiki Ramen,43.766998,-79.412222,Ramen Restaurant
2,Hillcrest Village,43.803762,-79.363452,The Keg,43.766579,-79.412131,Steakhouse
3,Hillcrest Village,43.803762,-79.363452,Cineplex Cinemas Empress Walk,43.768625,-79.412613,Movie Theater
4,Hillcrest Village,43.803762,-79.363452,Starbucks,43.768353,-79.413046,Coffee Shop


In [70]:
Borough_northyork_data_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",35,35,35,35,35,35
Bayview Village,35,35,35,35,35,35
"Bedford Park, Lawrence Manor East",35,35,35,35,35,35
"CFB Toronto, Downsview East",35,35,35,35,35,35
Don Mills North,35,35,35,35,35,35
Downsview Central,35,35,35,35,35,35
Downsview Northwest,35,35,35,35,35,35
Downsview West,35,35,35,35,35,35
"Downsview, North Park, Upwood Park",35,35,35,35,35,35


In [71]:
print('There are {} uniques categories.'.format(len(Borough_northyork_data_venues['Venue Category'].unique())))

There are 26 uniques categories.


## 3. Analyze Each Neighborhood

In [72]:
# one hot encoding
Northyork_onehot = pd.get_dummies(Borough_northyork_data_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Northyork_onehot['Neighborhood'] = Borough_northyork_data_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns1 = [Northyork_onehot.columns[-1]] + list(Northyork_onehot.columns[:-1])
Northyork_onehot = Northyork_onehot[fixed_columns1]

Northyork_onehot.head()

,Neighborhood,Arts & Crafts Store,Bubble Tea Shop,Café,Coffee Shop,Discount Store,Electronics Store,Fast Food Restaurant,Grocery Store,Hotel,...,Pet Store,Pizza Place,Plaza,Ramen Restaurant,Restaurant,Sandwich Place,Shopping Mall,Steakhouse,Sushi Restaurant,Vietnamese Restaurant
0,Hillcrest Village,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
Northyork_onehot.shape

(840, 27)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [74]:
Northyork_grouped = Northyork_onehot.groupby('Neighborhood').mean().reset_index()
Northyork_grouped

,Neighborhood,Arts & Crafts Store,Bubble Tea Shop,Café,Coffee Shop,Discount Store,Electronics Store,Fast Food Restaurant,Grocery Store,Hotel,...,Pet Store,Pizza Place,Plaza,Ramen Restaurant,Restaurant,Sandwich Place,Shopping Mall,Steakhouse,Sushi Restaurant,Vietnamese Restaurant
0,"Bathurst Manor, Downsview North, Wilson Heights",0.028571,0.028571,0.057143,0.057143,0.028571,0.028571,0.028571,0.028571,0.028571,...,0.028571,0.057143,0.028571,0.085714,0.057143,0.057143,0.028571,0.028571,0.085714,0.028571
1,Bayview Village,0.028571,0.028571,0.057143,0.057143,0.028571,0.028571,0.028571,0.028571,0.028571,...,0.028571,0.057143,0.028571,0.085714,0.057143,0.057143,0.028571,0.028571,0.085714,0.028571
2,"Bedford Park, Lawrence Manor East",0.028571,0.028571,0.057143,0.057143,0.028571,0.028571,0.028571,0.028571,0.028571,...,0.028571,0.057143,0.028571,0.085714,0.057143,0.057143,0.028571,0.028571,0.085714,0.028571
3,"CFB Toronto, Downsview East",0.028571,0.028571,0.057143,0.057143,0.028571,0.028571,0.028571,0.028571,0.028571,...,0.028571,0.057143,0.028571,0.085714,0.057143,0.057143,0.028571,0.028571,0.085714,0.028571
4,Don Mills North,0.028571,0.028571,0.057143,0.057143,0.028571,0.028571,0.028571,0.028571,0.028571,...,0.028571,0.057143,0.028571,0.085714,0.057143,0.057143,0.028571,0.028571,0.085714,0.028571
5,Downsview Central,0.028571,0.028571,0.057143,0.057143,0.028571,0.028571,0.028571,0.028571,0.028571,...,0.028571,0.057143,0.028571,0.085714,0.057143,0.057143,0.028571,0.028571,0.085714,0.028571
6,Downsview Northwest,0.028571,0.028571,0.057143,0.057143,0.028571,0.028571,0.028571,0.028571,0.028571,...,0.028571,0.057143,0.028571,0.085714,0.057143,0.057143,0.028571,0.028571,0.085714,0.028571
7,Downsview West,0.028571,0.028571,0.057143,0.057143,0.028571,0.028571,0.028571,0.028571,0.028571,...,0.028571,0.057143,0.028571,0.085714,0.057143,0.057143,0.028571,0.028571,0.085714,0.028571
8,"Downsview, North Park, Upwood Park",0.028571,0.028571,0.057143,0.057143,0.028571,0.028571,0.028571,0.028571,0.028571,...,0.028571,0.057143,0.028571,0.085714,0.057143,0.057143,0.028571,0.028571,0.085714,0.028571
9,"Emery, Humberlea",0.028571,0.028571,0.057143,0.057143,0.028571,0.028571,0.028571,0.028571,0.028571,...,0.028571,0.057143,0.028571,0.085714,0.057143,0.057143,0.028571,0.028571,0.085714,0.028571


In [75]:
Northyork_grouped.shape

(24, 27)

#### Let's print each neighborhood along with the top 5 most common venues

In [76]:
num_top_venues1 = 5

for hood in Northyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Northyork_grouped[Northyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Downsview North, Wilson Heights----
                       venue  freq
0           Sushi Restaurant  0.09
1           Ramen Restaurant  0.09
2                       Café  0.06
3                Coffee Shop  0.06
4             Sandwich Place  0.06
5                 Restaurant  0.06
6                Pizza Place  0.06
7        Arts & Crafts Store  0.03
8  Middle Eastern Restaurant  0.03
9                 Steakhouse  0.03


----Bayview Village----
                       venue  freq
0           Sushi Restaurant  0.09
1           Ramen Restaurant  0.09
2                       Café  0.06
3                Coffee Shop  0.06
4             Sandwich Place  0.06
5                 Restaurant  0.06
6                Pizza Place  0.06
7        Arts & Crafts Store  0.03
8  Middle Eastern Restaurant  0.03
9                 Steakhouse  0.03


----Bedford Park, Lawrence Manor East----
                       venue  freq
0           Sushi Restaurant  0.09
1           Ramen Restaurant  0.09

In [77]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted1 = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted1['Neighborhood'] = Northyork_grouped['Neighborhood']

for ind in np.arange(Northyork_grouped.shape[0]):
    neighborhoods_venues_sorted1.iloc[ind, 1:] = return_most_common_venues(Northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted1.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Downsview North, Wilson Heights",Ramen Restaurant,Sushi Restaurant,Café,Sandwich Place,Restaurant,Coffee Shop,Pizza Place,Vietnamese Restaurant,Ice Cream Shop,Bubble Tea Shop
1,Bayview Village,Ramen Restaurant,Sushi Restaurant,Café,Sandwich Place,Restaurant,Coffee Shop,Pizza Place,Vietnamese Restaurant,Ice Cream Shop,Bubble Tea Shop
2,"Bedford Park, Lawrence Manor East",Ramen Restaurant,Sushi Restaurant,Café,Sandwich Place,Restaurant,Coffee Shop,Pizza Place,Vietnamese Restaurant,Ice Cream Shop,Bubble Tea Shop
3,"CFB Toronto, Downsview East",Ramen Restaurant,Sushi Restaurant,Café,Sandwich Place,Restaurant,Coffee Shop,Pizza Place,Vietnamese Restaurant,Ice Cream Shop,Bubble Tea Shop
4,Don Mills North,Ramen Restaurant,Sushi Restaurant,Café,Sandwich Place,Restaurant,Coffee Shop,Pizza Place,Vietnamese Restaurant,Ice Cream Shop,Bubble Tea Shop


## 4. Cluster Neighborhoods

In [78]:
# set number of clusters
kclusters = 5

Northyork_grouped_clustering = Northyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Northyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [79]:
# add clustering labels
neighborhoods_venues_sorted1.insert(0, 'Cluster Labels', kmeans.labels_)

Northyork_merged = Borough_northyork_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Northyork_merged = Northyork_merged.join(neighborhoods_venues_sorted1.set_index('Neighborhood'), on='Neighborhood')

Northyork_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,0,Ramen Restaurant,Sushi Restaurant,Café,Sandwich Place,Restaurant,Coffee Shop,Pizza Place,Vietnamese Restaurant,Ice Cream Shop,Bubble Tea Shop
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0,Ramen Restaurant,Sushi Restaurant,Café,Sandwich Place,Restaurant,Coffee Shop,Pizza Place,Vietnamese Restaurant,Ice Cream Shop,Bubble Tea Shop
2,M2K,North York,Bayview Village,43.786947,-79.385975,0,Ramen Restaurant,Sushi Restaurant,Café,Sandwich Place,Restaurant,Coffee Shop,Pizza Place,Vietnamese Restaurant,Ice Cream Shop,Bubble Tea Shop
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,0,Ramen Restaurant,Sushi Restaurant,Café,Sandwich Place,Restaurant,Coffee Shop,Pizza Place,Vietnamese Restaurant,Ice Cream Shop,Bubble Tea Shop
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,0,Ramen Restaurant,Sushi Restaurant,Café,Sandwich Place,Restaurant,Coffee Shop,Pizza Place,Vietnamese Restaurant,Ice Cream Shop,Bubble Tea Shop


In [80]:
map_clusters1 = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Northyork_merged['Latitude'], Northyork_merged['Longitude'], Northyork_merged['Neighborhood'], Northyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters1)
       
map_clusters1

## 5. Examine Clusters

Now, examining each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster

#### Cluster 1

In [81]:
Northyork_merged.loc[Northyork_merged['Cluster Labels'] == 0, Northyork_merged.columns[[1] + list(range(5, Northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Ramen Restaurant,Sushi Restaurant,Café,Sandwich Place,Restaurant,Coffee Shop,Pizza Place,Vietnamese Restaurant,Ice Cream Shop,Bubble Tea Shop
1,North York,0,Ramen Restaurant,Sushi Restaurant,Café,Sandwich Place,Restaurant,Coffee Shop,Pizza Place,Vietnamese Restaurant,Ice Cream Shop,Bubble Tea Shop
2,North York,0,Ramen Restaurant,Sushi Restaurant,Café,Sandwich Place,Restaurant,Coffee Shop,Pizza Place,Vietnamese Restaurant,Ice Cream Shop,Bubble Tea Shop
3,North York,0,Ramen Restaurant,Sushi Restaurant,Café,Sandwich Place,Restaurant,Coffee Shop,Pizza Place,Vietnamese Restaurant,Ice Cream Shop,Bubble Tea Shop
4,North York,0,Ramen Restaurant,Sushi Restaurant,Café,Sandwich Place,Restaurant,Coffee Shop,Pizza Place,Vietnamese Restaurant,Ice Cream Shop,Bubble Tea Shop
5,North York,0,Ramen Restaurant,Sushi Restaurant,Café,Sandwich Place,Restaurant,Coffee Shop,Pizza Place,Vietnamese Restaurant,Ice Cream Shop,Bubble Tea Shop
6,North York,0,Ramen Restaurant,Sushi Restaurant,Café,Sandwich Place,Restaurant,Coffee Shop,Pizza Place,Vietnamese Restaurant,Ice Cream Shop,Bubble Tea Shop
7,North York,0,Ramen Restaurant,Sushi Restaurant,Café,Sandwich Place,Restaurant,Coffee Shop,Pizza Place,Vietnamese Restaurant,Ice Cream Shop,Bubble Tea Shop
8,North York,0,Ramen Restaurant,Sushi Restaurant,Café,Sandwich Place,Restaurant,Coffee Shop,Pizza Place,Vietnamese Restaurant,Ice Cream Shop,Bubble Tea Shop
9,North York,0,Ramen Restaurant,Sushi Restaurant,Café,Sandwich Place,Restaurant,Coffee Shop,Pizza Place,Vietnamese Restaurant,Ice Cream Shop,Bubble Tea Shop


#### Cluster 2

In [82]:
Northyork_merged.loc[Northyork_merged['Cluster Labels'] == 1, Northyork_merged.columns[[1] + list(range(5, Northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 3

In [83]:
Northyork_merged.loc[Northyork_merged['Cluster Labels'] == 2, Northyork_merged.columns[[1] + list(range(5, Northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 4

In [84]:
Northyork_merged.loc[Northyork_merged['Cluster Labels'] == 3, Northyork_merged.columns[[1] + list(range(5, Northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 5

In [85]:
Northyork_merged.loc[Northyork_merged['Cluster Labels'] == 4, Northyork_merged.columns[[1] + list(range(5, Northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
